<a href="https://colab.research.google.com/github/robertoricci/Curso-Data-Science-com-Python---Banco-MODAL/blob/main/Calculo_20_Empresas_pagadoras_de_Dividendos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance 

In [3]:
import pandas as pd
import yfinance as yf

In [7]:
#Lista de ibovespa,
symbol_list_ibrx = ['RRRP3',
 'ALPA4',
 'ABEV3',
 'AMER3',
 'ASAI3',
 'AZUL4',
 'B3SA3',
 'BIDI11',
 'BPAN4',
 'BBSE3',
 'BRML3',
 'BBDC3',
 'BBDC4',
 'BRAP4',
 'BBAS3',
 'BRKM5',
 'BRFS3',
 'BPAC11',
 'CRFB3',
 'CCRO3',
 'CMIG4',
 'CIEL3',
 'COGN3',
 'CPLE6',
 'CSAN3',
 'CPFE3',
 'CMIN3',
 'CVCB3',
 'CYRE3',
 'DXCO3',
 'ECOR3',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENBR3',
 'ENGI11',
 'ENEV3',
 'EGIE3',
 'EQTL3',
 'EZTC3',
 'FLRY3',
 'GGBR4',
 'GOAU4',
 'GOLL4',
 'NTCO3',
 'SOMA3',
 'HAPV3',
 'HYPE3',
 'IGTI11',
 'IRBR3',
 'ITSA4',
 'ITUB4',
 'JBSS3',
 'KLBN11',
 'LIGT3',
 'RENT3',
 'LCAM3',
 'LWSA3',
 'AMAR3',
 'LREN3',
 'MGLU3',
 'MRFG3',
 'CASH3',
 'BEEF3',
 'MOVI3',
 'MRVE3',
 'MULT3',
 'PCAR3',
 'PETR3',
 'PETR4',
 'PRIO3',
 'PETZ3',
 'PSSA3',
 'POSI3',
 'QUAL3',
 'RADL3',
 'RDOR3',
 'RAIL3',
 'SBSP3',
 'SANB11',
 'STBP3',
 'CSNA3',
 'SLCE3',
 'SULA11',
 'SUZB3',
 'TAEE11',
 'VIVT3',
 'TIMS3',
 'TOTS3',
 'UGPA3',
 'USIM5',
 'VALE3',
 'VIIA3',
 'VBBR3',
 'WEGE3',
 'YDUQ3']

In [99]:
symbol_list_ibrx = ['PETR4','YDUQ3']

In [16]:
df_scan = pd.DataFrame()
for ativo in symbol_list_ibrx:

    df2 =pd.DataFrame()
    chamada_api = yf.Ticker(ativo+'.SA')

    #pegar dados de 12 meses
    closes = chamada_api.history(period='12mo')['Close'].values
    dividendos = chamada_api.history(period='12mo')['Dividends'].values
    datas = list(chamada_api.history(period='12mo').index)
    

    df2['Closes'] = closes
    df2['Dividendos'] = dividendos
    df2['Datas'] = datas
    df2 = df2.set_index('Datas',drop = True)

    soma_dividendos = 0

    for k,v in df2.iterrows():
       #somas 
        #if k.year == 2021 or k.year == 2022:
        soma_dividendos += v['Dividendos']
                       #dividendos no ultimo ano    pegando o valor do ultimo fechamento
                       #caso queira para mais de e so dividir para qtd de ano
    dividend_yield = ((soma_dividendos/1) / df2['Closes'].values[-1] ) * 100

    ##df_scan[ativo] =  [dividend_yield]
    ##df_scan2[ativo] = [soma_dividendos]

    df_scan[ativo] = dividend_yield,soma_dividendos
#order pelo maior dividindo 
df_scan = df_scan.transpose().sort_values(by = 0,ascending=False)[:20]
#renomear colunas atualizando o dataframe 
df_scan.rename(columns = {0:'% DIV YIELD',1:'SOMA DIV Periodo' }, inplace= True)

In [17]:
#20 empresa que mais pagaram dividendos no ano selecionado
df_scan

,% DIV YIELD,SOMA DIV Periodo
PETR4,39.487473,11.546137
PETR3,35.658235,11.546137
MRFG3,35.002907,5.057920
BRAP4,33.023809,9.197131
BRKM5,25.978478,10.931744
GOAU4,22.100457,2.420000
USIM5,20.166982,1.917880
CPFE3,17.691401,5.441875
CMIN3,17.386349,0.876272
VALE3,17.261582,14.106165


In [32]:
#buscar dados fundamentalista das empresas
list_melhores = []
list_relacao_mm = []
list_precos = []
df_ativos =  pd.DataFrame()

for i in df_scan.index:
       #buscar dados do balanço
      chamada_api = yf.Ticker(i+'.SA')
      print('Ativo' + i)
      try:
            balanco_anual = chamada_api.balance_sheet
            balanco_trimestral = chamada_api.quarterly_balance_sheet
            dre_anual = chamada_api.financials
            dfc_anual = chamada_api.cashflow
           
            dre_trimestral = chamada_api.quarterly_financials
            dfc_trimestral = chamada_api.quarterly_cashflow
            
            if dre_anual.keys()[0].year >= dre_trimestral.keys()[0].year and len(balanco_anual) > 0:
                balanco = balanco_anual
                dre = dre_anual
                dfc = dfc_anual
                variacao_receita = int(dre.loc['Total Revenue'][0] > dre.loc['Total Revenue'][1])
                variacao_ebit =    int(dre.loc['Ebit'][0] > dre.loc['Ebit'][1])
                variacao_caixa =   int(dfc.loc['Change In Cash'][0] > dfc.loc['Change In Cash'][1])
                variacao_divida =  int(balanco.loc['Total Current Liabilities'][0] < balanco.loc['Total Current Liabilities'][1])
                
                df_ativos[ativo] = variacao_divida,variacao_caixa,variacao_ebit,variacao_receita,dre_anual.keys()[0]
                
                soma_fundamentos = variacao_caixa + variacao_ebit + variacao_receita + variacao_divida
                if soma_fundamentos >= 2:
                    print(ativo,variacao_divida,variacao_caixa,variacao_ebit,variacao_receita,dre_anual.keys()[0])
                    list_melhores.append(ativo)
                    #list_relacao_mm.append(abs(hist_precos[-1] / np.mean(hist_precos[-40:])-1))
                    #list_precos.append(round(hist_precos[-1],2))
            else:
                continue
      except:
        print('Erro' + i)
        continue


AtivoPETR4
YDUQ3 1 0 1 1 2021-12-31 00:00:00
AtivoPETR3
YDUQ3 1 0 1 1 2021-12-31 00:00:00
AtivoMRFG3
YDUQ3 0 0 1 1 2021-12-31 00:00:00
AtivoBRAP4
YDUQ3 0 1 0 1 2021-12-31 00:00:00
AtivoBRKM5
YDUQ3 1 0 1 1 2021-12-31 00:00:00
AtivoGOAU4
YDUQ3 0 0 1 1 2021-12-31 00:00:00
AtivoUSIM5
AtivoCPFE3
YDUQ3 1 0 1 1 2021-12-31 00:00:00
AtivoCMIN3
YDUQ3 0 1 1 1 2021-12-31 00:00:00
AtivoVALE3
AtivoCPLE6
YDUQ3 1 0 1 1 2021-12-31 00:00:00
AtivoCMIG4
YDUQ3 0 0 1 1 2021-12-31 00:00:00
AtivoGGBR4
YDUQ3 0 0 1 1 2021-12-31 00:00:00
AtivoDXCO3
YDUQ3 0 0 1 1 2021-12-31 00:00:00
AtivoJBSS3
YDUQ3 0 0 1 1 2021-12-31 00:00:00
AtivoENBR3
YDUQ3 1 0 1 1 2021-12-31 00:00:00
AtivoPOSI3
AtivoSANB11
AtivoCSNA3
YDUQ3 0 0 1 1 2021-12-31 00:00:00
AtivoEGIE3
YDUQ3 0 0 1 1 2021-12-31 00:00:00


In [44]:
i='PETR4'

In [45]:
 chamada_api = yf.Ticker(i+'.SA')

In [47]:
balanco_anual = chamada_api.balance_sheet

In [48]:
balanco_anual

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Intangible Assets,1.675600e+10,7.755300e+10,7.823700e+10,1.008400e+10
Total Liab,5.833700e+11,6.762690e+11,6.268740e+11,5.769300e+11
Total Stockholder Equity,3.873290e+11,3.084100e+11,2.955410e+11,2.772250e+11
Minority Interest,2.252000e+09,2.740000e+09,3.596000e+09,6.318000e+09
Other Current Liab,2.000400e+10,1.800000e+10,2.943300e+10,3.620900e+10
Total Assets,9.729510e+11,9.874190e+11,9.260110e+11,8.604730e+11
Common Stock,2.054320e+11,2.054320e+11,2.054320e+11,2.054320e+11
Other Current Assets,2.267200e+10,1.047600e+10,1.858400e+10,2.058500e+10
Retained Earnings,1.642440e+11,1.275120e+11,1.248290e+11,9.536400e+10
Other Liab,1.712700e+11,1.987980e+11,2.007050e+11,1.672820e+11


In [55]:
 balanco_trimestral = chamada_api.quarterly_balance_sheet

In [56]:
balanco_trimestral

,2021-12-31,2021-09-30,2021-06-30,2021-03-31
Intangible Assets,1.675600e+10,1.647600e+10,7.751400e+10,7.746100e+10
Total Liab,5.833700e+11,5.990000e+11,5.790570e+11,6.780380e+11
Total Stockholder Equity,3.873290e+11,3.690770e+11,3.560520e+11,3.133950e+11
Minority Interest,2.252000e+09,6.612000e+09,6.338000e+09,6.850000e+09
Other Current Liab,2.000400e+10,2.808100e+10,1.466100e+10,1.853200e+10
Total Assets,9.729510e+11,9.746890e+11,9.414470e+11,9.982830e+11
Common Stock,2.054320e+11,2.054320e+11,2.054320e+11,2.054320e+11
Other Current Assets,2.267200e+10,3.088600e+10,3.482800e+10,2.438600e+10
Retained Earnings,1.642440e+11,1.651100e+11,1.656730e+11,1.286790e+11
Other Liab,1.712700e+11,1.857510e+11,1.867660e+11,1.986020e+11


In [38]:
  balanco_trimestral = chamada_api.quarterly_balance_sheet

In [39]:
balanco_trimestral

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [54]:
balanco_anual.loc['Total Current Liabilities'][0] 


134913000000.0

In [57]:
 dre_anual = chamada_api.financials

In [58]:
dre_anual

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Research Development,3033000000.0,1819000000.0,2268000000.0,2345000000.0
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,151575000000.0,37000000.0,47242000000.0,38967000000.0
Minority Interest,2252000000.0,2740000000.0,3596000000.0,6318000000.0
Net Income,106668000000.0,7108000000.0,40137000000.0,25779000000.0
Selling General Administrative,29146000000.0,30545000000.0,26114000000.0,22084000000.0
Gross Profit,219637000000.0,123962000000.0,122105000000.0,118687000000.0
Ebit,187491000000.0,80307000000.0,77770000000.0,70668000000.0
Operating Income,187491000000.0,80307000000.0,77770000000.0,70668000000.0
Other Operating Expenses,6037000000.0,-7274000000.0,13707000000.0,16192000000.0
